In [1]:
from google.colab import files
uploaded=files.upload()

Saving EnglishTenseTruncatedDataset.xlsx to EnglishTenseTruncatedDataset (1).xlsx


In [2]:
for file_name in uploaded.keys():
  print(f"Uploaded file {file_name}")

Uploaded file EnglishTenseTruncatedDataset (1).xlsx


In [41]:
import pandas as pd
file_path= "EnglishTenseTruncatedDataset.xlsx"
df = pd.read_excel(file_path)
df.head()

,Sentence,Truncated_Sentence
0,by 2050 ai architects will have designed selfc...,by 2050 ai architects will
1,in the future sustainable transportation optio...,in the future sustainable transportation optio...
2,china has been actively involved in peacekeepi...,china has been actively involved in
3,educational diversity is a hallmark of foreign...,educational diversity is a hallmark
4,the coach substituted an underperforming player,the coach substituted an


### **GPT 2**

In [19]:
# Install necessary libraries
!pip install transformers torch pandas tqdm

# Import required libraries
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import pandas as pd
from tqdm import tqdm

# Step 1: Define Hyperparameters for Allowing Natural Completions
hyperparameters = {
    'max_length': 50,      # Increase max length to ensure it reaches a punctuation mark
    'temperature': 0.7,    # Balanced temperature for coherent completions
    'top_k': 50,           # Reasonable top-k for good diversity
    'top_p': 0.9,          # Nucleus sampling for focused completions
    'no_repeat_ngram_size': 2  # Prevent repeating phrases
}

# Step 2: Load the GPT-2 Model and Tokenizer
model_name = "gpt2"  # You can switch to 'gpt2-medium', 'gpt2-large', or 'gpt2-xl' for larger models
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Set pad_token_id to eos_token_id to avoid warnings
model.config.pad_token_id = model.config.eos_token_id
tokenizer.pad_token = tokenizer.eos_token  # Set the pad_token as eos_token

# Step 3: Function to Generate Completions and Truncate to Full Stop
def generate_until_full_stop(truncated_sentence, model, tokenizer, params=hyperparameters):
    # Encode the prompt
    input_ids = tokenizer.encode(truncated_sentence, return_tensors='pt')

    # Generate the completion
    output = model.generate(
        input_ids,
        max_length=params['max_length'],
        temperature=params['temperature'],
        top_k=params['top_k'],
        top_p=params['top_p'],
        no_repeat_ngram_size=params['no_repeat_ngram_size'],
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id
    )

    # Decode the generated text
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

    # Remove the original prompt to get only the new completion
    completion = generated_text[len(truncated_sentence):].strip()

    # Truncate at the first occurrence of a full stop, exclamation mark, or question mark
    for end_marker in ['.', '!', '?']:
        if end_marker in completion:
            completion = completion.split(end_marker)[0] + end_marker
            break

    return completion

# Step 4: Load the Dataset
file_path = "EnglishTenseTruncatedDataset (3).xlsx"  # Update this path as needed
df = pd.read_excel(file_path)

# Step 5: Generate Completions with a Progress Bar and Truncate at Full Stop
df['GPT2_xj'] = df['Truncated_Sentence'].progress_apply(lambda x: generate_until_full_stop(x, model, tokenizer))

# Step 6: Create the Full Sentence Column
df['GPT2_complete'] = df.apply(lambda row: f"{row['Truncated_Sentence']} {row['GPT2_xj']}", axis=1)

# Step 7: Display the First Few Rows to Verify
df.head()

# Step 8: Save the Updated Dataset
df.to_excel("EnglishTenseWithTruncatedGPT2Completions.xlsx", index=False)

# Download the updated file
from google.colab import files
files.download("EnglishTenseWithTruncatedGPT2Completions.xlsx")


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
100%|██████████| 19/19 [00:34<00:00,  1.82s/it]


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

GPT J

In [20]:
# Install necessary libraries
!pip install transformers torch pandas tqdm

# Import required libraries
from transformers import AutoModelForCausalLM, AutoTokenizer
import pandas as pd
from tqdm import tqdm

# Set up tqdm to work with Pandas
tqdm.pandas()  # This adds progress bar support to Pandas' apply methods

# Step 1: Define Adjusted Hyperparameters for More Coherent Completions
adjusted_hyperparameters = {
    'max_length': 40,      # Increase max length to ensure it reaches a punctuation mark
    'temperature': 0.5,    # Lower temperature for more deterministic output
    'top_k': 50,           # Slightly higher top-k for more diversity
    'top_p': 0.95,         # Slightly higher top-p for more natural sampling
    'no_repeat_ngram_size': 2  # Prevent repeating phrases
}

# Step 2: Load the GPT-J Model and Tokenizer
model_name = "EleutherAI/gpt-j-6B"  # Use the base GPT-J model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Set pad_token_id to eos_token_id to avoid warnings
model.config.pad_token_id = tokenizer.eos_token_id
tokenizer.pad_token = tokenizer.eos_token  # Set the pad_token as eos_token

# Step 3: Function to Generate Completions and Truncate at Logical Stopping Point
def generate_until_full_stop(truncated_sentence, model, tokenizer, params=adjusted_hyperparameters):
    # Encode the prompt
    input_ids = tokenizer.encode(truncated_sentence, return_tensors='pt')

    # Generate the completion
    output = model.generate(
        input_ids,
        max_length=params['max_length'],
        temperature=params['temperature'],
        top_k=params['top_k'],
        top_p=params['top_p'],
        no_repeat_ngram_size=params['no_repeat_ngram_size'],
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id
    )

    # Decode the generated text
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

    # Remove the original prompt to get only the new completion
    completion = generated_text[len(truncated_sentence):].strip()

    # Truncate at the first occurrence of a full stop, exclamation mark, or question mark
    for end_marker in ['.', '!', '?']:
        if end_marker in completion:
            completion = completion.split(end_marker)[0] + end_marker
            break

    return completion

# Step 4: Load the Dataset
file_path = "EnglishTenseTruncatedDataset (3).xlsx"  # Update this path as needed
df = pd.read_excel(file_path)

# Step 5: Generate Completions with a Progress Bar and Truncate at Full Stop
df['GPT_J_xj'] = df['Truncated_Sentence'].progress_apply(lambda x: generate_until_full_stop(x, model, tokenizer))

# Step 6: Create the Full Sentence Column
df['GPT_J_complete'] = df.apply(lambda row: f"{row['Truncated_Sentence']} {row['GPT_J_xj']}", axis=1)

# Step 7: Display the First Few Rows to Verify
df.head()

# Step 8: Save the Updated Dataset
df.to_excel("EnglishTenseWithGPTJCompletions_Coherent.xlsx", index=False)

# Download the updated file
from google.colab import files
files.download("EnglishTenseWithGPTJCompletions_Coherent.xlsx")


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
100%|██████████| 19/19 [11:30<00:00, 36.34s/it]


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

GPT Neo

In [33]:
# Install necessary libraries
!pip install transformers torch pandas tqdm

# Import required libraries
from transformers import GPTNeoForCausalLM, GPT2Tokenizer
import pandas as pd
from tqdm import tqdm

# Set up tqdm to work with Pandas
tqdm.pandas()  # This adds progress bar support to Pandas' apply methods

# Step 1: Define Hyperparameters for GPT-Neo
gpt_neo_hyperparameters = {
    'max_length': 60,      # Allow more room for completions
    'temperature': 0.7,    # Slight randomness for diversity
    'top_p': 0.9,          # Nucleus sampling for coherent yet diverse completions
    'repetition_penalty': 1.2  # Discourage repetition
}

# Step 2: Load the GPT-Neo Model and Tokenizer
model_name = "EleutherAI/gpt-neo-1.3B"  # You can also use "EleutherAI/gpt-neo-2.7B"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPTNeoForCausalLM.from_pretrained(model_name)

# Step 3: Function to Generate Completions with GPT-Neo
def generate_gpt_neo_completion(truncated_sentence, model, tokenizer, params=gpt_neo_hyperparameters):
    # Encode the input sentence directly
    input_ids = tokenizer.encode(truncated_sentence, return_tensors='pt')

    # Generate the completion
    output = model.generate(
        input_ids,
        max_length=params['max_length'],
        temperature=params['temperature'],
        top_p=params['top_p'],
        repetition_penalty=params['repetition_penalty'],
        do_sample=True
    )

    # Decode the generated text
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

    # Remove the original prompt to get only the new completion
    completion = generated_text[len(truncated_sentence):].strip()

    # Truncate at the first occurrence of a full stop, exclamation mark, or question mark
    for end_marker in ['.', '!', '?']:
        if end_marker in completion:
            completion = completion.split(end_marker)[0] + end_marker
            break

    return completion

# Step 4: Load the Dataset
file_path = "EnglishTenseTruncatedDataset (3).xlsx"  # Update this path as needed
df = pd.read_excel(file_path)

# Step 5: Generate Completions with GPT-Neo and the Refined Hyperparameters
df['GPTNeo_xj'] = df['Truncated_Sentence'].progress_apply(lambda x: generate_gpt_neo_completion(x, model, tokenizer))

# Step 6: Create the Full Sentence Column
df['GPTNeo_complete'] = df.apply(lambda row: f"{row['Truncated_Sentence']} {row['GPTNeo_xj']}", axis=1)

# Step 7: Display the First Few Rows to Verify
df.head()

# Step 8: Save the Updated Dataset
df.to_excel("EnglishTenseWithGPTNeoCompletions.xlsx", index=False)

# Download the updated file
from google.colab import files
files.download("EnglishTenseWithGPTNeoCompletions.xlsx")


tokenizer_config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.35k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/5.31G [00:00<?, ?B/s]

  0%|          | 0/19 [00:00<?, ?it/s]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
 11%|█         | 2/19 [00:14<02:03,  7.25s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
 16%|█▌        | 3/19 [00:28<02:43, 10.22s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
 21%|██        | 4/19 [00:42<02:54, 11.66s/it]The attention mask and the pad token id were not set. As a consequence,

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

BLOOM

In [48]:
# Install necessary libraries
!pip install transformers torch pandas tqdm

# Import required libraries
from transformers import AutoModelForCausalLM, AutoTokenizer
import pandas as pd
from tqdm import tqdm

# Load the BLOOM Model and Tokenizer
model_name = 'bigscience/bloom-560m'  # Options: 'bigscience/bloom', 'bigscience/bloom-1b7', etc.
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Set pad_token_id to eos_token_id to avoid warnings
model.config.pad_token_id = model.config.eos_token_id
tokenizer.pad_token = tokenizer.eos_token

# Define Hyperparameters
hyperparameters = {
    'max_length': 50,                # Max length of the generated sequence
    'temperature': 0.7,              # Adjusts randomness
    'top_k': 50,                     # Limits the next token selection to the top_k options
    'top_p': 0.9,                    # Nucleus sampling
    'no_repeat_ngram_size': 2,       # Prevent repeating phrases
    'do_sample': True                # Enable sampling
}

# Function to Generate Completions
def generate_until_full_stop_bloom(truncated_sentence, model, tokenizer, params=hyperparameters):
    # Prepare the input text
    input_text = truncated_sentence.strip()
    input_ids = tokenizer.encode(input_text, return_tensors='pt')

    # Generate the completion
    output_ids = model.generate(
        input_ids=input_ids,
        max_length=input_ids.shape[1] + params['max_length'],
        temperature=params['temperature'],
        top_k=params['top_k'],
        top_p=params['top_p'],
        no_repeat_ngram_size=params['no_repeat_ngram_size'],
        do_sample=params['do_sample'],
        pad_token_id=tokenizer.eos_token_id
    )

    # Decode the generated text
    generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    # Remove the original prompt to get only the new completion
    completion = generated_text[len(truncated_sentence):].strip()

    # Truncate at the first occurrence of a punctuation mark
    for end_marker in ['.', '!', '?']:
        if end_marker in completion:
            completion = completion.split(end_marker)[0] + end_marker
            break

    return completion

# Load the Dataset
file_path = "EnglishTenseTruncatedDataset (4).xlsx"  # Update this path as needed
df = pd.read_excel(file_path)

# Ensure the progress_apply function works properly
tqdm.pandas()

# Generate Completions
df['BLOOM_xj'] = df['Truncated_Sentence'].progress_apply(
    lambda x: generate_until_full_stop_bloom(x, model, tokenizer)
)

# Create the Full Sentence Column
df['BLOOM_complete'] = df.apply(
    lambda row: f"{row['Truncated_Sentence']} {row['BLOOM_xj']}", axis=1
)

# Save the Updated Dataset
df.to_excel("EnglishTenseWithTruncatedBLOOMCompletions.xlsx", index=False)

# Download the updated file (if using Colab)
from google.colab import files
files.download("EnglishTenseWithTruncatedBLOOMCompletions.xlsx")


tokenizer_config.json:   0%|          | 0.00/222 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/693 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

100%|██████████| 10/10 [01:14<00:00,  7.45s/it]


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Not fixed

OPT

In [66]:
# Install necessary libraries
!pip install transformers torch pandas tqdm

# Import required libraries
from transformers import AutoTokenizer, AutoModelForCausalLM
import pandas as pd
from tqdm import tqdm

# Load the OPT Model and Tokenizer
model_name = 'facebook/opt-1.3b'  # Options: 'opt-125m', 'opt-350m', etc.
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Move model to GPU if available
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)

# Define Hyperparameters
hyperparameters = {
    'max_length': 50,
    'temperature': 0.7,
    'top_k': 50,
    'top_p': 0.9,
    'do_sample': True,
    'no_repeat_ngram_size': 2,
    'early_stopping': True
}

# Function to Generate Completions
def generate_completion_opt(truncated_sentence, model, tokenizer, params=hyperparameters):
    input_ids = tokenizer.encode(truncated_sentence, return_tensors='pt').to(device)

    output_ids = model.generate(
        input_ids=input_ids,
        max_length=input_ids.shape[1] + params['max_length'],
        temperature=params['temperature'],
        top_k=params['top_k'],
        top_p=params['top_p'],
        do_sample=params['do_sample'],
        no_repeat_ngram_size=params['no_repeat_ngram_size'],
        early_stopping=params['early_stopping'],
        pad_token_id=tokenizer.eos_token_id
    )

    generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    completion = generated_text[len(truncated_sentence):].strip()

    # Truncate at the first punctuation mark
    for end_marker in ['.', '!', '?']:
        if end_marker in completion:
            completion = completion.split(end_marker)[0] + end_marker
            break

    return completion

# Load the Dataset
file_path = "EnglishTenseTruncatedDataset (4).xlsx"
df = pd.read_excel(file_path)

# Generate Completions
tqdm.pandas()
df['OPT_xj'] = df['Truncated_Sentence'].progress_apply(
    lambda x: generate_completion_opt(x, model, tokenizer)
)

# Create the Full Sentence Column
df['OPT_complete'] = df['Truncated_Sentence'] + ' ' + df['OPT_xj']

# Save the Updated Dataset
df.to_excel("EnglishTenseWithOPTCompletions.xlsx", index=False)

from google.colab import files
files.download("EnglishTenseWithOPTCompletions.xlsx")


  0%|          | 0/10 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:615: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
100%|██████████| 10/10 [02:18<00:00, 13.86s/it]


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

LLama

In [6]:
# Install necessary libraries
!pip install transformers torch pandas tqdm sentencepiece

# Import necessary libraries
import torch
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
import pandas as pd
from tqdm import tqdm

# Check if GPU is available, if so, use it
device = 0 if torch.cuda.is_available() else -1  # 0 for GPU, -1 for CPU

# Initialize the pipeline and set the device
pipe = pipeline("text-generation", model="meta-llama/Llama-2-7b-hf", device=device)


# Load the Llama 2 Model and Tokenizer using the pipeline for GPU
model_name = 'meta-llama/Llama-2-7b'  # Use the 'hf' version
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Move model to GPU if available
model.to(device)

# Define Hyperparameters
hyperparameters = {
    'max_length': 30,  # Reduce from 50 to 30
    'temperature': 0.7,
    'top_k': 50,
    'top_p': 0.9,
    'do_sample': True,
    'no_repeat_ngram_size': 2,
    'early_stopping': True
}

# Function to Generate Completions
def generate_completion_llama(truncated_sentence, model, tokenizer, params=hyperparameters):
    input_ids = tokenizer.encode(truncated_sentence, return_tensors='pt').to(device)

    output_ids = model.generate(
        input_ids=input_ids,
        max_length=input_ids.shape[1] + params['max_length'],
        temperature=params['temperature'],
        top_k=params['top_k'],
        top_p=params['top_p'],
        do_sample=params['do_sample'],
        no_repeat_ngram_size=params['no_repeat_ngram_size'],
        early_stopping=params['early_stopping'],
        pad_token_id=tokenizer.eos_token_id
    )

    generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    completion = generated_text[len(truncated_sentence):].strip()

    # Truncate at the first punctuation mark
    for end_marker in ['.', '!', '?']:
        if end_marker in completion:
            completion = completion.split(end_marker)[0] + end_marker
            break

    return completion

# Load the Dataset
file_path = "EnglishTenseTruncatedDataset.xlsx"  # Ensure this file is uploaded in Colab
df = pd.read_excel(file_path)

# Generate Completions
tqdm.pandas()
df['Llama2_xj'] = df['Truncated_Sentence'].progress_apply(
    lambda x: generate_completion_llama(x, model, tokenizer)
)

# Create the Full Sentence Column
df['Llama2_complete'] = df['Truncated_Sentence'] + ' ' + df['Llama2_xj']

# Save the Updated Dataset
df.to_excel("EnglishTenseWithLlama2Completions.xlsx", index=False)

# Download the file from Colab
from google.colab import files
files.download("EnglishTenseWithLlama2Completions.xlsx")


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 500.00 MiB. GPU 0 has a total capacity of 14.75 GiB of which 1.06 MiB is free. Process 49818 has 14.74 GiB memory in use. Of the allocated memory 14.65 GiB is allocated by PyTorch, and 1.43 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

CTRL

In [73]:
# Install necessary libraries
!pip install transformers torch pandas tqdm

# Import required libraries
from transformers import CTRLTokenizer, CTRLLMHeadModel
import pandas as pd
from tqdm import tqdm
import torch

# Load the CTRL Model and Tokenizer
model_name = 'ctrl'
tokenizer = CTRLTokenizer.from_pretrained(model_name)
model = CTRLLMHeadModel.from_pretrained(model_name)

# Move model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Define Hyperparameters
hyperparameters = {
    'max_length': 50,
    'temperature': 0.95,
    'top_k': 50,
    'top_p': 0.9,
    'do_sample': True,
    'num_return_sequences': 1
}

# Function to Generate Completions
def generate_completion_ctrl(truncated_sentence, model, tokenizer, params=hyperparameters):
    # CTRL requires control codes; we'll use 'Links' as a neutral code
    control_code = 'Links'
    input_text = control_code + ' ' + truncated_sentence.strip()
    input_ids = tokenizer.encode(input_text, return_tensors='pt').to(device)
    input_length = input_ids.shape[1]

    # Generate the completion
    output_ids = model.generate(
        input_ids=input_ids,
        max_length=input_length + params['max_length'],
        temperature=params['temperature'],
        top_k=params['top_k'],
        top_p=params['top_p'],
        do_sample=params['do_sample'],
        num_return_sequences=params['num_return_sequences'],
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id
    )

    # Decode the generated text
    generated_text = tokenizer.decode(output_ids[0][input_length:], skip_special_tokens=True)
    completion = generated_text.strip()

    # Truncate at the first punctuation mark
    for end_marker in ['.', '!', '?']:
        if end_marker in completion:
            completion = completion.split(end_marker)[0] + end_marker
            break

    return completion

# Load the Dataset
file_path = "EnglishTenseTruncatedDataset (4).xlsx"
df = pd.read_excel(file_path)

# Generate Completions
tqdm.pandas()
df['CTRL_xj'] = df['Truncated_Sentence'].progress_apply(
    lambda x: generate_completion_ctrl(x, model, tokenizer)
)

# Create the Full Sentence Column
df['CTRL_complete'] = df['Truncated_Sentence'] + ' ' + df['CTRL_xj']

# Save the Updated Dataset
df.to_excel("EnglishTenseWithCTRLCompletions.xlsx", index=False)



from google.colab import files
files.download("EnglishTenseWithCTRLCompletions.xlsx")


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
100%|██████████| 10/10 [03:58<00:00, 23.84s/it]


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [71]:
pip install sentencepiece sacremoses

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 8.4 MB/s eta 0:00:00


In [59]:
from google.colab import files
files.download("EnglishTenseWithXLNetCompletions.xlsx")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [4]:
from huggingface_hub import login

# This will prompt you to input your Hugging Face token
login()
